In [5]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [6]:
CLIENT_ID = 'QZ0KLTTKJKEBNEXGRGZZMW14BEZWRV00NX54Z3GV4CULO12S' # your Foursquare ID
CLIENT_SECRET = 'CVN1IR5PBKDGUIEQYP3G0RCQTFGWJEN2FFBVQV2GMMZHLUA1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QZ0KLTTKJKEBNEXGRGZZMW14BEZWRV00NX54Z3GV4CULO12S
CLIENT_SECRET:CVN1IR5PBKDGUIEQYP3G0RCQTFGWJEN2FFBVQV2GMMZHLUA1


In [ ]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

In [ ]:
search_query = ''
radius = 5000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

In [ ]:
venues = results['response']['venues']
#users = results_users['response']['users']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

In [ ]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df_Ny = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_Ny['categories'] = df_Ny.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_Ny.columns = [column.split('.')[-1] for column in df_Ny.columns]

df_Ny

In [ ]:
address_tor = 'Bayview Extension, Toronto, TO'

geolocator = Nominatim(user_agent="foursquare_agent")
location_tor = geolocator.geocode(address_tor)
latitude_tor = location_tor.latitude
longitude_tor = location_tor.longitude
print(latitude_tor, longitude_tor)

In [ ]:
search_query = ''
radius = 5000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_tor, longitude_tor, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

In [ ]:
venues = results['response']['venues']
#users = results_users['response']['users']

# tranform venues into a dataframe
dataframe_tor = json_normalize(venues)
dataframe_tor

In [ ]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe_tor.columns if col.startswith('location.')] + ['id']
df_Tr = dataframe_tor.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_Tr['categories'] = df_Tr.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_Tr.columns = [column.split('.')[-1] for column in df_Tr.columns]

df_Tr

In [ ]:
df_N = df_Ny['categories']
df_T = df_Tr['categories']

df_N = pd.get_dummies(df_N)
df_T = pd.get_dummies(df_T)

In [ ]:
# set number of clusters
from sklearn.cluster import KMeans

kclusters = 5



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_N)

# check cluster labels generated for each row in the dataframe
set(kmeans.labels_[0:10])

In [ ]:
df_N

In [ ]:

#df_Tr['crossStreet'].isnull().sum()
df_Tr.groupby(['crossStreet'])['crossStreet'].count()

In [ ]:
k_means = KMeans(init="k-means++", n_clusters=4, n_init=12)
k_means.fit(df_N)

k_means_labels = k_means.labels_
k_means_labels

In [ ]:
k_means = KMeans(init="k-means++", n_clusters=4, n_init=12)
k_means.fit(df_Tr)

k_means_labels = k_means.labels_
k_means_labels